In [58]:
from docx import Document
from docx.oxml import OxmlElement
import re
from lxml import etree
from latex2mathml.converter import convert
import mathml2omml

In [73]:
from docx import Document
from docx.oxml import OxmlElement
from lxml import etree
import re
from latex2mathml.converter import convert
import mathml2omml

def create_omml_element(latex_code):
    try:
        mathml = convert(latex_code)
        # print(f"MathML: {mathml}")
    except Exception as e:
        raise ValueError(f"Error converting LaTeX to MathML: {e}")

    try:
        omml = mathml2omml.convert(mathml)
        # print(f"OMML (before namespace fix): {omml}")
    except Exception as e:
        raise ValueError(f"Error converting MathML to OMML: {e}")

    if 'xmlns:m=' not in omml:
        omml = omml.replace(
            "<m:oMath",
            '<m:oMath xmlns:m="http://schemas.openxmlformats.org/officeDocument/2006/math"'
        )

    try:
        omml_element = etree.fromstring(omml.encode('utf-8'))
    except Exception as e:
        raise ValueError(f"Error parsing OMML: {e}")

    return omml_element
def insert_omml(paragraph, latex_code):
    omml_element = create_omml_element(latex_code)
    paragraph._element.append(omml_element)

def process_content_to_word(data):
    """
    Process content into a Word document with LaTeX math equations, formatted text, and proper line breaks.
    """
    document = Document()
    paragraph = document.add_paragraph()  # Start with an initial paragraph

    for item in data:
        text = item['insert']
        attributes = item.get('attributes', {})
        latex_pattern = r"\[:(.*?)\]"  # Match LaTeX patterns
        parts = re.split(latex_pattern, text)

        for i, part in enumerate(parts):
            if i % 2 == 0:
                # Regular text
                run = paragraph.add_run(part.rstrip("\n"))  # Remove trailing \n for processing
                if attributes.get('bold'):
                    run.bold = True
                if attributes.get('underline'):
                    run.underline = True
                if attributes.get('italic'):
                    run.italic = True
            else:
                # LaTeX math code
                insert_omml(paragraph, part)

        # If the text ends with a newline, create a new paragraph
        if text.endswith("\n"):
            paragraph = document.add_paragraph()  # Start a new paragraph

    document.save("output.docx")

# Example content
content = [
    {'insert': '수학공식 [:\\frac{1}{2} \\sum_{1}^{n} n^2]과 '},
    {'insert': '다음 텍스트 \n', 'attributes': {'bold': True, 'underline': True, 'italic': True}},
    {'insert': '다음줄 텍스트 \n', 'attributes': {'bold': True, 'underline': True, 'italic': True}},
]

# Process the content into a Word document
process_content_to_word(content)

MathML: <math xmlns="http://www.w3.org/1998/Math/MathML" display="inline"><mrow><mfrac><mrow><mn>1</mn></mrow><mrow><mn>2</mn></mrow></mfrac><msubsup><mo>&#x02211;</mo><mrow><mn>1</mn></mrow><mrow><mi>n</mi></mrow></msubsup><msup><mi>n</mi><mn>2</mn></msup></mrow></math>
OMML (before namespace fix): <m:oMath><m:box><m:e><m:f><m:num><m:box><m:e><m:r><m:rPr><m:sty m:val="p"/></m:rPr><m:t>1</m:t></m:r></m:e></m:box></m:num><m:den><m:box><m:e><m:r><m:rPr><m:sty m:val="p"/></m:rPr><m:t>2</m:t></m:r></m:e></m:box></m:den></m:f><m:nary><m:naryPr><m:chr m:val="∑"/><m:limLoc m:val="undOvr"/></m:naryPr><m:sub><m:box><m:e><m:r><m:rPr><m:sty m:val="p"/></m:rPr><m:t>1</m:t></m:r></m:e></m:box></m:sub><m:sup><m:box><m:e><m:r><m:rPr><m:sty m:val="i"/></m:rPr><m:t>n</m:t></m:r></m:e></m:box></m:sup><m:e><m:sSup><m:e><m:r><m:rPr><m:sty m:val="i"/></m:rPr><m:t>n</m:t></m:r></m:e><m:sup><m:r><m:rPr><m:sty m:val="p"/></m:rPr><m:t>2</m:t></m:r></m:sup></m:sSup></m:e></m:nary></m:e></m:box></m:oMath>
